In [1]:
# from serial.main import train_maddpg
from parallel.main import train_maddpg

min_scale = 5
middle_scale = 10
max_scale = 20

train_maddpg(middle_scale, episodes=500)

--- Environment Ready ---
Agents: 10
Dims -> Agent Obs: 50, Global State: 280, Action: 5
Params -> K_local: 4, M_global: 36
Episode finished after 13 steps
Ep 0 | Avg Reward: 0.2 | Success: 0.72 | Fitness: 0.96 | distance: 1412.27, time: 1737.82
Episode finished after 21 steps
Ep 1 | Avg Reward: 0.1 | Success: 0.75 | Fitness: 0.96 | distance: 1562.93, time: 1696.82
Ep 2 | Avg Reward: 0.2 | Success: 0.75 | Fitness: 0.88 | distance: 1368.76, time: 1455.61
Ep 3 | Avg Reward: 0.1 | Success: 0.72 | Fitness: 0.80 | distance: 1278.92, time: 1110.24
Episode finished after 21 steps
Ep 4 | Avg Reward: -0.1 | Success: 0.69 | Fitness: 0.97 | distance: 1420.64, time: 1692.46
Episode finished after 13 steps
Ep 5 | Avg Reward: 0.4 | Success: 0.83 | Fitness: 0.97 | distance: 1398.72, time: 1372.64
Episode finished after 21 steps
Ep 6 | Avg Reward: 0.1 | Success: 0.75 | Fitness: 0.96 | distance: 1499.58, time: 1636.02
Episode finished after 16 steps
Ep 7 | Avg Reward: 0.0 | Success: 0.67 | Fitness: 0.9

KeyboardInterrupt: 